# EP 2 - Relações entre Pokémons

## Desenvolvedores
- Carolina Gimenez
- Eduarda Medeiros
- Gustavo Pagnotta

# Imports

In [1]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 8.5 MB/s eta 0:00:00


In [2]:
import os
from neo4j import GraphDatabase
import json
import re

# Credenciais

In [3]:
senha = "Y_kgind87Zp_lKAFNsQSzwb0MdLo0Z6KMo8MW4jXjNo"
url = "neo4j+s://8fd1cf81.databases.neo4j.io"
usuario = "neo4j"

# Criação dos scripts com base no arquivo JSON

In [4]:
def criacao_base():
  with open("/content/pokemon_data.json", "r") as file:
      pokemon_data = json.load(file)

  # Gerar comandos CREATE
  create_statements = []
  connection_statements = []

  for pokemon in pokemon_data:
      types = []
      name = pokemon['name'].lower()
      if "♀" in name or "♂" in name or "\'" in name or "." in name or "-" in name or " " in name:
        name = re.sub(r'[^\w\s]', '', name)
        name = name.replace(" ", "_").replace("(", "").replace(")", "")

      types = pokemon['types'].split(",")
      evolutions_list = pokemon['evolution']

      # Criação do Pokemon
      create_pokemon = f"CREATE ({name}:POKEMON {{id: {pokemon['id']}, name: \"{pokemon['name']}\", height: \"{pokemon['height']}\", weight: \"{pokemon['weight']}\", color: \"{pokemon['color']}\"}})\n"
      create_statements.append(create_pokemon)

      # Criação dos tipos
      for ptype in types:
          connection_statements.append(f"CREATE ({name})-[:HAS_TYPE]->({ptype.lower()})\n")

      # Criação das evoluções
      for evolutions in evolutions_list:
          evolution_name = evolutions['name'].lower()

          if "♀" in evolution_name or "♂" in evolution_name or "\'" in evolution_name or "." in evolution_name or "-" in evolution_name:
            evolution_name = re.sub(r'[^\w\s]', '', evolution_name)
            evolution_name = evolution_name.replace(" ", "_").replace("(", "").replace(")", "")

          connection_statements.append(f"CREATE ({name})-[:EVOLUTION]->({evolution_name})\n")

  return "".join(create_statements) + "\n" + "".join(connection_statements)

# Criação da base

In [6]:
class CriarRelacoes:
  def __init__(self, uri, user, password):
    self.driver = GraphDatabase.driver(uri, auth=(user, password))

  def close(self):
    self.driver.close()

  def carrega_base(self):
    with self.driver.session() as session:
      session.write_transaction(self._zera_base)
      session.write_transaction(self._cria_relacoes)

  @staticmethod
  def _zera_base(tx):
    tx.run("""
MATCH (n)
DETACH DELETE n
""")

  @staticmethod
  def _cria_relacoes(tx):
    base = criacao_base()

    tipos_criacao = """
      CREATE (water:TYPE {type: 'Water'})
      CREATE (fairy:TYPE {type: 'Fairy'})
      CREATE (rock:TYPE {type: 'Rock'})
      CREATE (normal:TYPE {type: 'Normal'})
      CREATE (steel:TYPE {type: 'Steel'})
      CREATE (grass:TYPE {type: 'Grass'})
      CREATE (electric:TYPE {type: 'Electric'})
      CREATE (fighting:TYPE {type: 'Fighting'})
      CREATE (poison:TYPE {type: 'Poison'})
      CREATE (psychic:TYPE {type: 'Psychic'})
      CREATE (ice:TYPE {type: 'Ice'})
      CREATE (ghost:TYPE {type: 'Ghost'})
      CREATE (flying:TYPE {type: 'Flying'})
      CREATE (dragon:TYPE {type: 'Dragon'})
      CREATE (ground:TYPE {type: 'Ground'})
      CREATE (fire:TYPE {type: 'Fire'})
      CREATE (dark:TYPE {type: 'Dark'})
      CREATE (bug:TYPE {type: 'Bug'})
    """

    fraqueza_tipos = """
    CREATE (water)-[:WEAKNESS]->(electric)
    CREATE (water)-[:WEAKNESS]->(grass)
    CREATE (fire)-[:WEAKNESS]->(water)
    CREATE (fire)-[:WEAKNESS]->(rock)
    CREATE (fire)-[:WEAKNESS]->(ground)
    CREATE (grass)-[:WEAKNESS]->(fire)
    CREATE (grass)-[:WEAKNESS]->(flying)
    CREATE (grass)-[:WEAKNESS]->(bug)
    CREATE (grass)-[:WEAKNESS]->(poison)
    CREATE (grass)-[:WEAKNESS]->(ice)
    CREATE (electric)-[:WEAKNESS]->(ground)
    CREATE (ground)-[:WEAKNESS]->(water)
    CREATE (ground)-[:WEAKNESS]->(ice)
    CREATE (ground)-[:WEAKNESS]->(grass)
    CREATE (flying)-[:WEAKNESS]->(electric)
    CREATE (flying)-[:WEAKNESS]->(ice)
    CREATE (flying)-[:WEAKNESS]->(rock)
    CREATE (rock)-[:WEAKNESS]->(water)
    CREATE (rock)-[:WEAKNESS]->(grass)
    CREATE (rock)-[:WEAKNESS]->(fighting)
    CREATE (rock)-[:WEAKNESS]->(ground)
    CREATE (rock)-[:WEAKNESS]->(steel)
    CREATE (steel)-[:WEAKNESS]->(fire)
    CREATE (steel)-[:WEAKNESS]->(fighting)
    CREATE (steel)-[:WEAKNESS]->(ground)
    CREATE (bug)-[:WEAKNESS]->(fire)
    CREATE (bug)-[:WEAKNESS]->(flying)
    CREATE (bug)-[:WEAKNESS]->(rock)
    CREATE (ice)-[:WEAKNESS]->(fire)
    CREATE (ice)-[:WEAKNESS]->(fighting)
    CREATE (ice)-[:WEAKNESS]->(rock)
    CREATE (ice)-[:WEAKNESS]->(steel)
    CREATE (fighting)-[:WEAKNESS]->(flying)
    CREATE (fighting)-[:WEAKNESS]->(psychic)
    CREATE (fighting)-[:WEAKNESS]->(fairy)
    CREATE (poison)-[:WEAKNESS]->(ground)
    CREATE (poison)-[:WEAKNESS]->(psychic)
    CREATE (psychic)-[:WEAKNESS]->(bug)
    CREATE (psychic)-[:WEAKNESS]->(ghost)
    CREATE (psychic)-[:WEAKNESS]->(dark)
    CREATE (ghost)-[:WEAKNESS]->(ghost)
    CREATE (ghost)-[:WEAKNESS]->(dark)
    CREATE (dragon)-[:WEAKNESS]->(ice)
    CREATE (dragon)-[:WEAKNESS]->(dragon)
    CREATE (dragon)-[:WEAKNESS]->(fairy)
    CREATE (dark)-[:WEAKNESS]->(fighting)
    CREATE (dark)-[:WEAKNESS]->(bug)
    CREATE (dark)-[:WEAKNESS]->(fairy)
    CREATE (fairy)-[:WEAKNESS]->(steel)
    CREATE (fairy)-[:WEAKNESS]->(poison)
    CREATE (normal)-[:WEAKNESS]->(fighting)
    """

    tx.run(tipos_criacao + "\n" + base + "\n" + fraqueza_tipos)

db = CriarRelacoes(url, usuario, senha)
db.carrega_base()
db.close()
print("Base carregada")

<ipython-input-6-3575854ad9d7>:10: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._zera_base)
<ipython-input-6-3575854ad9d7>:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._cria_relacoes)


Base carregada


# Consultas

In [7]:
class Neo4jConnection:
  def __init__(self, uri, user, pwd):
    self.__uri = uri
    self.__user = user
    self.__pwd = pwd
    self.__driver = None
    try:
        self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
    except Exception as e:
        print("Failed to create the driver:", e)

  def close(self):
    if self.__driver is not None:
        self.__driver.close()

  def query(self, query, parameters=None, db=None):
    assert self.__driver is not None, "Driver not initialized!"
    session = None
    response = None
    try:
      session = self.__driver.session(database=db) if db is not None else self.__driver.session()
      response = list(session.run(query, parameters))
    except Exception as e:
      print("Query failed:", e)
    finally:
      if session is not None:
        session.close()
    return response

conn = Neo4jConnection(uri=url,
                       user=usuario,
                       pwd=senha)

def consultar(query):
  return conn.query(query)
  # for line in conn.query(query):
  #   print(dict(line))

## Quais Pokémons podem atacar um Pikachu pela sua fraqueza cujo o peso é mais de 10kg?

In [8]:
print("Pokémons que atacam um Pikachu pela sua fraqueza com peso maior que 10kg:\n")

consultar("""
MATCH (pikachu:POKEMON {name: "Pikachu"})-[:HAS_TYPE]->(t1:TYPE)-[:WEAKNESS]->(t2:TYPE)<-[:HAS_TYPE]-(p:POKEMON)
WHERE toFloat(replace(p.weight, ' kg', '')) > 10
RETURN p.name as ataca_pikachu, p.weight as peso
""")

Pokémons que atacam um Pikachu pela sua fraqueza com peso maior que 10kg:



[<Record ataca_pikachu='Sandshrew' peso='12.0 kg'>,
 <Record ataca_pikachu='Sandslash' peso='29.5 kg'>,
 <Record ataca_pikachu='Nidoqueen' peso='60.0 kg'>,
 <Record ataca_pikachu='Nidoking' peso='62.0 kg'>,
 <Record ataca_pikachu='Dugtrio' peso='33.3 kg'>,
 <Record ataca_pikachu='Geodude' peso='20.0 kg'>,
 <Record ataca_pikachu='Graveler' peso='105.0 kg'>,
 <Record ataca_pikachu='Golem' peso='300.0 kg'>,
 <Record ataca_pikachu='Onix' peso='210.0 kg'>,
 <Record ataca_pikachu='Marowak' peso='45.0 kg'>,
 <Record ataca_pikachu='Rhyhorn' peso='115.0 kg'>,
 <Record ataca_pikachu='Rhydon' peso='120.0 kg'>,
 <Record ataca_pikachu='Quagsire' peso='75.0 kg'>,
 <Record ataca_pikachu='Gligar' peso='64.8 kg'>,
 <Record ataca_pikachu='Steelix' peso='400.0 kg'>,
 <Record ataca_pikachu='Piloswine' peso='55.8 kg'>,
 <Record ataca_pikachu='Phanpy' peso='33.5 kg'>,
 <Record ataca_pikachu='Donphan' peso='120.0 kg'>,
 <Record ataca_pikachu='Larvitar' peso='72.0 kg'>,
 <Record ataca_pikachu='Pupitar' peso='

## Qual a cor mais comum de um Pokémon que é atacado pelo tipo gelo?

In [9]:
print("\nCor mais comum de um Pokémon atacado pelo tipo gelo:\n")

consultar("""
MATCH (pokemon:POKEMON)-[:HAS_TYPE]->(t1:TYPE)-[:WEAKNESS]->(t2:TYPE {type: "Ice"})
RETURN pokemon.color, COUNT(pokemon.color) AS color_count
ORDER BY color_count DESC
LIMIT 1
""")


Cor mais comum de um Pokémon atacado pelo tipo gelo:



[<Record pokemon.color='blue' color_count=93>]

## Quantas segundas e terceiras evoluções que fazem um Pokémon no mínimo dobrar de peso?

In [11]:
print("\nQuantidade de Segundas e Terceiras evoluções fazem um Pokémon no mínimo dobrar de peso:\n")

consultar("""
MATCH (p1:POKEMON)-[:EVOLUTION]->(p2:POKEMON)
WHERE toFloat(replace(p2.weight, ' kg', '')) >= 2 * toFloat(replace(p1.weight, ' kg', ''))
WITH p1, p2
RETURN COUNT(DISTINCT p2) AS total_evolucoes
""")


Quantidade de Segundas e Terceiras evoluções fazem um Pokémon no mínimo dobrar de peso:



[<Record total_evolucoes=412>]